In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
sales = pd.read_csv("kc_house_data_train.csv", index_col = 0)
holdout = pd.read_csv("kc_house_data_test_features.csv", index_col = 0)
filledrooms = pd.read_csv("bedroomsfilled.csv", index_col = 0)
extra = pd.read_csv("zip_code_extra_data.csv", index_col = 0)

In [3]:
# def filled_data(df,df2):
#     index=df.loc[df.bedrooms == 0].index.to_list()
#     df.drop(df.index[[index]], inplace=True)
#     frames= [df, df2]
#     train = pd.concat(frames)
#     return train
# sales=filled_data(sales,filledrooms)

In [4]:
print(sales.shape)
sales.head()

(17290, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [5]:
extra.reset_index(inplace = True)

In [18]:
extra["zipcode"].nunique()

AttributeError: 'int' object has no attribute 'to_list'

In [7]:
#sales[sales["zipcode"]==98116]

In [8]:
# sales["month"] = pd.DatetimeIndex(sales["date"]).month
# sales.drop(columns=["id","date"], inplace = True)

In [9]:
# df=sales.merge(extra, how="right", on="zipcode")
#extra.set_index('zipcode', inplace=True)
training_data = pd.merge(sales, extra, how='right', on='zipcode')

In [10]:
sales["zipcode"].nunique()

70

In [11]:
training_data.shape

(17755, 75)